<a href="https://colab.research.google.com/github/sergiochaza/sqlite/blob/main/SQL_TCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import pandas as pd

In [2]:
#Conectar a una base de datos en memoria
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

#Crear la tabla de cuentas
cursor.execute('''
    CREATE TABLE cuentas (
        id INTEGER PRIMARY KEY,
        nombre TEXT NOT NULL,
        saldo REAL NOR NULL
    )
''')

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('cuentas',)]


In [4]:
def transferir_dinero(origen,destino,monto):

    #procedimiento para transferir dinero entre cuentas.
    #utiliza TCL: BEGIN TRANSACTION, COMMIT, ROLLBACK

    try:
      #Iniciar la transacción
      cursor.execute("BEGIN TRANSACTION;")


      #Verificar saldo suficiente
      cursor.execute("SELECT saldo FROM cuentas WHERE id=?",(origen,))
      saldo_origen = cursor.fetchone()[0]

      if saldo_origen < monto:
        raise Exception("Saldo insuficiente")

      #Restar saldo de la cuenta de origen
      cursor.execute("UPDATE cuentas SET saldo = saldo - ? WHERE id = ?",(monto,origen))


      #Sumar saldo a la cuenta de destino
      cursor.execute("UPDATE cuentas SET saldo = saldo + ? WHERE id =?", (monto,destino))


      #Confirmar la transaccion
      conn.commit()
      print(f"Transferencia de ${monto} completada con éxito")


    except Exception as e:
      #Recertir la transacción en caso de error
      conn.rollback()
      print(f"Error al realizar la transferencia: {e}")



In [5]:
#Insertar cuentas de prueba
cursor.executemany("""
  INSERT INTO cuentas (id,nombre,saldo) VALUES (?,?,?)
  """, [(1,'Cuenta A', 1000.00), (2,'Cuenta B', 500.00)])
conn.commit()

In [6]:
#Consultar los saldos antes de la transacción
cursor.execute("SELECT * FROM cuentas")
print(cursor.fetchall())

[(1, 'Cuenta A', 1000.0), (2, 'Cuenta B', 500.0)]


In [7]:
#Prueba la transferencia
transferir_dinero(1,2,300)

Transferencia de $300 completada con éxito


In [8]:
#Consultar los saldos después de la transacción
cursor.execute("SELECT * FROM cuentas")
print(cursor.fetchall())

[(1, 'Cuenta A', 700.0), (2, 'Cuenta B', 800.0)]


In [9]:
#Prueba la transferencia
transferir_dinero(1,2,200)

Transferencia de $200 completada con éxito


In [10]:
#Consultar los saldos después de la transacción
cursor.execute("SELECT * FROM cuentas")
print(cursor.fetchall())

[(1, 'Cuenta A', 500.0), (2, 'Cuenta B', 1000.0)]


In [11]:
#Cierro la conexión
conn.close()